In [1]:
import os
import pandas as pd
from pandas.tseries.offsets import BDay
from pandas.tseries.offsets import CustomBusinessDay
from pandas.tseries.holiday import USFederalHolidayCalendar

## Get and massage portfolio data

In [2]:
ticker_histories_filename = os.path.join("input", "ticker_histories.csv")
ticker_histories_df = pd.read_csv(ticker_histories_filename)
ticker_histories_df["datetime"] = pd.to_datetime(
    ticker_histories_df["timestamp"], unit="ms"
).dt.tz_localize(None)
ticker_histories_df.set_index("datetime", inplace=True)
ticker_histories_df.drop("timestamp", axis=1, inplace=True)
ticker_histories_df.sort_index(inplace=True)
ticker_histories_df

,ticker,open,high,low,close,volume,vwap,transactions
datetime,,,,,,,,
2020-04-01 04:00:00,AMZN,96.6485,97.2480,94.6500,95.3850,82437500.0,95.8985,141417
2020-04-01 04:00:00,MSFT,153.0000,157.7500,150.8200,152.1100,57969926.0,153.7717,531817
2020-04-01 04:00:00,AAPL,61.6250,62.1800,59.7825,60.2275,176046552.0,60.9782,460605
2020-04-01 04:00:00,GOOG,56.1000,56.4845,54.8725,55.2810,46883460.0,55.6376,71423
2020-04-01 04:00:00,NVDA,6.3913,6.5383,6.0320,6.0768,656913120.0,6.2836,185913
...,...,...,...,...,...,...,...,...
2025-03-13 04:00:00,AAPL,215.9500,216.8394,208.4200,209.6800,59752532.0,212.0980,768619
2025-03-13 04:00:00,GOOG,167.9800,168.1200,164.0700,164.7300,14841144.0,165.4467,253905
2025-03-13 04:00:00,AMZN,198.1650,198.8799,191.8200,193.8900,39427497.0,194.3506,588340


In [13]:
adj_close_df = ticker_histories_df.reset_index().pivot(
    index="datetime", columns="ticker", values="close"
)
adj_close_df.index = pd.DatetimeIndex(adj_close_df.index)
adj_close_df.index = pd.DatetimeIndex(
    [dt.replace(hour=17, minute=0, second=0) for dt in adj_close_df.index]
)
adj_close_df.tail()

ticker,AAPL,AMZN,GOOG,MSFT,NVDA,TSLA
2025-03-07 17:00:00,239.07,199.25,175.75,393.31,112.69,262.67
2025-03-10 17:00:00,227.48,194.54,167.81,380.16,106.98,222.15
2025-03-11 17:00:00,220.84,196.59,165.98,380.45,108.76,230.58
2025-03-12 17:00:00,216.98,198.89,169.00,383.27,115.74,248.09
2025-03-13 17:00:00,209.68,193.89,164.73,378.77,115.58,240.68


In [9]:
def future_business_day(start_date, business_days_ahead):
    """
    Calculates the business date a specified number of business days in the future,
    skipping US federal holidays.

    Args:
        start_date (pd.Timestamp or str): The starting date.
        business_days_ahead (int): The number of business days in the future.

    Returns:
        pd.Timestamp: The calculated future business date.
    """
    cal = USFederalHolidayCalendar()
    holidays = cal.holidays()
    cbd = CustomBusinessDay(holidays=holidays)
    return start_date + (cbd * business_days_ahead)

In [5]:
def create_business_day_range(start_date, num_days):
    """
    Creates a range of business days starting from a given date, skipping US federal holidays.

    Args:
        start_date (pd.Timestamp or str): The starting date.
        num_days (int): The number of business days to generate.

    Returns:
        pd.DatetimeIndex: A DatetimeIndex containing the range of business days.
    """
    cal = USFederalHolidayCalendar()
    holidays = cal.holidays()
    cbd = CustomBusinessDay(holidays=holidays)
    return pd.date_range(start=start_date, periods=num_days, freq=cbd)

In [14]:
start_days = create_business_day_range(pd.Timestamp("2025-01-13"), 20)
for start_day in start_days:
    end_day = future_business_day(start_day, 20)
    print(start_day, "to", end_day)

2025-01-13 00:00:00 to 2025-02-11 00:00:00
2025-01-14 00:00:00 to 2025-02-12 00:00:00
2025-01-15 00:00:00 to 2025-02-13 00:00:00
2025-01-16 00:00:00 to 2025-02-14 00:00:00
2025-01-17 00:00:00 to 2025-02-18 00:00:00
2025-01-21 00:00:00 to 2025-02-19 00:00:00
2025-01-22 00:00:00 to 2025-02-20 00:00:00
2025-01-23 00:00:00 to 2025-02-21 00:00:00
2025-01-24 00:00:00 to 2025-02-24 00:00:00
2025-01-27 00:00:00 to 2025-02-25 00:00:00
2025-01-28 00:00:00 to 2025-02-26 00:00:00
2025-01-29 00:00:00 to 2025-02-27 00:00:00
2025-01-30 00:00:00 to 2025-02-28 00:00:00
2025-01-31 00:00:00 to 2025-03-03 00:00:00
2025-02-03 00:00:00 to 2025-03-04 00:00:00
2025-02-04 00:00:00 to 2025-03-05 00:00:00
2025-02-05 00:00:00 to 2025-03-06 00:00:00
2025-02-06 00:00:00 to 2025-03-07 00:00:00
2025-02-07 00:00:00 to 2025-03-10 00:00:00
2025-02-10 00:00:00 to 2025-03-11 00:00:00


In [15]:
histories = {}
start_days = create_business_day_range(pd.Timestamp("2025-01-13"), 20)
for start_day in start_days:
    end_day = future_business_day(start_day, 20)
    histories[(start_day, end_day)] = adj_close_df.loc[start_day:end_day]